# Short interest rate model calibration

In the earlier chapters, we have discussed simulating Hull-White model. That exercise gave a primer on how to use the model classes. There the model parameters were assumed to be given. However in practice, the model parameters need to calibrated from market data. Typically instruments such as swaptions, caps or floors and their market prices / volatilities are taken as inputs. Then the model parameters are fit in such a way that the model prices for these options are close enough. The goodness of fit depends, apart from the choice of the numerical methods, on the type of model itself. This is because models such as Hull-White 1 factor cannot fit some of the humped volatility term structures observed in the market. Never the less, Hull-White is usually a good starting point to understand calibration process.

Here we will discuss Hull-White model in detail. Then we will also show how the same procedure can be applied to calibrate other short rate models. We will assume the quotes to be at-the-money (ATM) log-normal volatilities. In the later section, we will extend to normal volatilities.

In [1]:
from QuantLib import *
from collections import namedtuple
import math
from pandas import DataFrame

#### Hull-White 1-Factor Model 

Hull-White model was one of the first practical exogenous models that attempted to fit to the market interest rate term structures. The model is described as:

$$ dr_t = (\theta(t) - a r_t) dt + \sigma dW_t $$

where $a$ is the mean reversion constant, $\sigma$ is the volatility parameter. The parameter $\theta(t)$ is chosen in order to fit the input term structure of interest rates. 

What is the "right" value for parameters $a$ and $\sigma$? This is the question that we address by calibrating to market instruments.

In [2]:
today = Date(15, February, 2002);
settlement= Date(19, February, 2002);
Settings.instance().evaluationDate = today;
term_structure = YieldTermStructureHandle(
    FlatForward(settlement,0.04875825,Actual365Fixed())
    )
index = Euribor1Y(term_structure)

In this example we are going to calibrate given the starting tenor, months to maturity, and the swaption volatilities as shown below. 

In [3]:
CalibrationData = namedtuple("CalibrationData", 
                             "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
        CalibrationData(2, 4, 0.1108),
        CalibrationData(3, 3, 0.1070),
        CalibrationData(4, 2, 0.1021),
        CalibrationData(5, 1, 0.1000 )]

In order to make the code succinct in the various examples, we will create two functions. Function `create_swaption_helpers` takes all the swaption data, the index such as `Euribor1Y`, the term structure and the pricing engine, and returns a list of `SwaptionHelper` objects. The `calibration_report` evaluates the calibration by comparing the model price and implied volatilities with the Black price and market volatilities.

In [4]:
def create_swaption_helpers(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = Period(1, Years)
    fixed_leg_daycounter = Actual360()
    floating_leg_daycounter = Actual360()
    for d in data:
        vol_handle = QuoteHandle(SimpleQuote(d.volatility))
        helper = SwaptionHelper(Period(d.start, Years),
                                   Period(d.length, Years),
                                   vol_handle,
                                   index,
                                   fixed_leg_tenor,
                                   fixed_leg_daycounter,
                                   floating_leg_daycounter,
                                   term_structure
                                   )
        helper.setPricingEngine(engine)
        swaptions.append(helper)
    return swaptions    


def calibration_report(swaptions, data):
    columns = ["Model Price", "Market Price", "Implied Vol", "Market Vol",
               "Rel Error Price", "Rel Error Vols"]
    report_data = []
    cum_err = 0.0
    cum_err2 = 0.0
    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = data[i].volatility
        black_price = s.blackPrice(market_vol)
        rel_error = model_price/black_price - 1.0
        implied_vol = s.impliedVolatility(model_price,
                                          1e-5, 50, 0.0, 0.50)
        rel_error2 = implied_vol/market_vol-1.0
        cum_err += rel_error*rel_error
        cum_err2 += rel_error2*rel_error2
        
        report_data.append((model_price, black_price, implied_vol,
                            market_vol, rel_error, rel_error2))
    print("Cumulative Error Price: %7.5f" % math.sqrt(cum_err))
    print("Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))
    return DataFrame(report_data,columns= columns, index=['']*len(report_data))

##### Calibrating Reversion and Volaitility

Here we use the `JamshidianSwaptionEngine` to value the swaptions as part of calibration. The `JamshidianSwaptionEngine` requires one-factor affine models as input. For other interest rate models, we need a pricing engine that is more suited to those models.

In [5]:
model = HullWhite(term_structure);
engine = JamshidianSwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)

optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.04642, sigma = 0.00580


In [6]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.11583
Cumulative Error Vols : 0.11614


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008775,0.009485,0.106198,0.1148,-0.074854,-0.074928
,0.009669,0.010078,0.106292,0.1108,-0.040610,-0.040688
,0.008663,0.008716,0.106343,0.1070,-0.006138,-0.006138
,0.006490,0.006226,0.106442,0.1021,0.042367,0.042525
,0.003542,0.003323,0.106612,0.1000,0.065817,0.066122


#### Calibrating Volatility With Fixed Reversion

There are times when we need to calibrate with one parameter held fixed. QuantLib allows you to perform calibration with constraints. However, this ability is not exposed in the SWIG wrappers as of version 1.6. I have created a [github issue](https://github.com/lballabio/quantlib-old/issues/336) and provided a patch to address this issue. This patch has been merged into QuantLib-SWIG version 1.7. If you are using version lower than 1.7, you will need this patch to execute the following cells. Below, the model is calibrated with a fixed reversion value of 5%.

The following code is similar to the Hull-White calibration, except we initialize the constrained model with given values. In the calibrate method, we provide a list of boolean with constraints `[True, False]`, meaning that the first parameter `a` is constrained where as the second `sigma` is not constrained.

In [7]:
constrained_model = HullWhite(term_structure, 0.05, 0.001);
engine = JamshidianSwaptionEngine(constrained_model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)

optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
constrained_model.calibrate(swaptions, optimization_method, 
                            end_criteria, NoConstraint(), 
                            [], [True, False])
a, sigma = constrained_model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.05000, sigma = 0.00586


In [8]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.11584
Cumulative Error Vols : 0.11615


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008776,0.009485,0.106212,0.1148,-0.074738,-0.074812
,0.009668,0.010078,0.106284,0.1108,-0.040682,-0.040761
,0.008662,0.008716,0.106330,0.1070,-0.006261,-0.006261
,0.006490,0.006226,0.106436,0.1021,0.042311,0.042469
,0.003542,0.003323,0.106625,0.1000,0.065946,0.066252


#### Black Karasinski Model

The Black Karasinski model is described as:

$$ d\ln(r_t) = (\theta_t - a \ln(r_t)) dt + \sigma dW_t $$

Black-Karasinski is not an affine model, and hence we cannot use the `JamshidianSwaptionEngine`. In order to calibrate, we use the `TreeSwaptionEngine` which will work with all short rate models. The calibration procedure is shown below. 

In [9]:
model = BlackKarasinski(term_structure);
engine = TreeSwaptionEngine(model, 100)
swaptions = create_swaption_helpers(data, index, term_structure, engine)

optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma =  model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.03902, sigma = 0.11695


In [10]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.12132
Cumulative Error Vols : 0.12163


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008717,0.009485,0.105497,0.1148,-0.080954,-0.081033
,0.009670,0.010078,0.106309,0.1108,-0.040453,-0.040531
,0.008679,0.008716,0.106540,0.1070,-0.004297,-0.004297
,0.006503,0.006226,0.106656,0.1021,0.044457,0.044623
,0.003547,0.003323,0.106765,0.1000,0.067333,0.067646


#### G2++ Model

As a final example, let us look at a calibration example of the 2-factor G2++ model.
$$ dr_t = \varphi(t) + x_t + y_t $$ 
    
where $ x_t $ and $ y_t $ are defined by

$$
\begin{eqnarray}
dx_t &=& -a x_t dt + \sigma dW^1_t  \\
dy_t &=& -b y_t dt + \eta dW^2_t \\
\left<dW^1_t dW^2_t\right> &=& \rho dt 
\end{eqnarray}
$$

Once again, we use the `TreeSwaptionEngine` to value the swaptions in the calibration step. One can also use `G2SwaptionEngine` and `FdG2SwaptionEngine`. But the calibration times, and accuracy can vary depending on the choice of parameters.

In [11]:
model = G2(term_structure);
engine = TreeSwaptionEngine(model, 25)
# engine = ql.G2SwaptionEngine(model, 10, 400)
# engine = ql.FdG2SwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)

optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(1000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma, b, eta, rho = model.params()
print("a = %6.5f, sigma = %6.5f, b = %6.5f, eta = %6.5f, rho = %6.5f " % \
      (a, sigma, b, eta, rho))

a = 0.03942, sigma = 0.00473, b = 0.04720, eta = 0.00301, rho = 0.03865 


In [12]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.12241
Cumulative Error Vols : 0.12272


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008704,0.009485,0.105333,0.1148,-0.082383,-0.082464
,0.009672,0.010078,0.106322,0.1108,-0.040334,-0.040412
,0.008676,0.008716,0.106510,0.1070,-0.004583,-0.004583
,0.006503,0.006226,0.106647,0.1021,0.044365,0.044531
,0.003548,0.003323,0.106800,0.1000,0.067681,0.067996


#### Calibrating to Normal Volatilities 

In certain markets in Europe and Japan for instance have had negative interest rates in the recent past for some of the tenors of the yield curve. The lognormal volatility quotes used above are inconsistent with negative rates and it is generally a practice to quote normal volaitilities in this case. The ``SwaptionHelperPtr`` used above with lognormal volatilities can be modified for normal volatilities by setting the ``VolatilityType`` parameter.


The full `C++` syntax for the `SwaptionHelper` object is as shown below:


    SwaptionHelperPtr(const Date& exerciseDate, const Period& length,
                  const Handle<Quote>& volatility,
                  const IborIndexPtr& index,
                  const Period& fixedLegTenor,
                  const DayCounter& fixedLegDayCounter,
                  const DayCounter& floatingLegDayCounter,
                  const Handle<YieldTermStructure>& termStructure,
                  CalibrationHelper::CalibrationErrorType errorType
                            = CalibrationHelper::RelativePriceError,
                  const Real strike = Null<Real>(),
                  const Real nominal = 1.0,
                  const VolatilityType type = ShiftedLognormal,
                  const Real shift = 0.0) 


In the above examples, we did not pass any of the optional arguments for `errorType`, `strike`, `nominal` and `type` specifying the `VolatilityType`. One can set the optional `type` parameter to change from log-normal volatilities to normal volatilities. 

A function to create swaption helpers with `normal` volatilities is shown below:

In [13]:
def create_swaption_helpers_normal(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = Period(1, Years)
    fixed_leg_daycounter = Actual360()
    floating_leg_daycounter = Actual360()
    for d in data:
        vol_handle = QuoteHandle(SimpleQuote(d.volatility))
        helper= SwaptionHelper(Period(d.start, Years),
                               Period(d.length, Years),
                               vol_handle,
                               index,
                               fixed_leg_tenor,
                               fixed_leg_daycounter,
                               floating_leg_daycounter,
                               term_structure,
                               CalibrationHelper.RelativePriceError,
                               nullDouble(),
                               1.0,
                               Normal
                               )
        helper.setPricingEngine(engine)
        swaptions.append(helper)
    return swaptions    


Now, we can call the ``create_swaption_helpers_normal`` instead to constructions swaptions with normal volatilities and pass it to the calibration routine to determine the model parameters.

In [14]:
model = HullWhite(term_structure);
engine = JamshidianSwaptionEngine(model)
swaptions = create_swaption_helpers_normal(data, index, term_structure, engine)

optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.04595, sigma = 0.11868


#### Conclusion

In this chapter, we saw some simple examples of calibrating the interest rate models to the swaption volatilities. We looked at setting up different interest rate models and discussed both lognormal and normal volatilities. 